In [2]:
# Шаг 1. Выгружаем с БД таблицы и создаем датафрейм

import psycopg2 as psycopg
import pandas as pd

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20240325_54955bf804",
    "user": "mle_20240325_54955bf804",
    "password": "6e3f607018b444f69359510efb12da90",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)
print(df)

         id customer_id begin_date   end_date            type   
0        17  8191-XWSZG 2015-10-01        NaT        One year  \
1        59  3957-SQXML 2017-04-01        NaT        Two year   
2       147  6837-BJYDQ 2019-11-01        NaT        One year   
3       482  0486-LGCCH 2019-03-01        NaT        Two year   
4         1  7590-VHVEG 2020-01-01        NaT  Month-to-month   
...     ...         ...        ...        ...             ...   
7038  12998  2823-LKABH 2018-08-01        NaT  Month-to-month   
7039  12999  8775-CEBBJ 2019-02-01 2019-11-01  Month-to-month   
7040  13000  0550-DCXLH 2019-01-01        NaT  Month-to-month   
7041  13001  9281-CEDRU 2014-06-01        NaT        Two year   
7042  13002  2235-DWLJU 2019-08-01        NaT  Month-to-month   

     paperless_billing             payment_method  monthly_charges   
0                   No               Mailed check            20.65  \
1                   No    Credit card (automatic)            24.95   
2        

In [3]:
# Шаг 2. Создаем артефакт "columns.txt"
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(','.join(df.columns))

In [4]:
# Шаг 3. Посчитаем различные статистики
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
	# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts().to_dict()
    print(column_stat)
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
	# обновите словарь stats
    stats.update(column_stat)


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()
stats["unique_customers_number"] = df["customer_id"].nunique()
stats["end_date_nan"] = df["end_date"].isnull().sum()
print(stats)

{'Month-to-month': 3875, 'Two year': 1695, 'One year': 1473}
{'Yes': 4171, 'No': 2872}
{'Fiber optic': 3096, 'DSL': 2421}
{'No': 3498, 'Yes': 2019}
{'No': 3088, 'Yes': 2429}
{'No': 3095, 'Yes': 2422}
{'No': 3473, 'Yes': 2044}
{'No': 2810, 'Yes': 2707}
{'No': 2785, 'Yes': 2732}
{'Male': 3555, 'Female': 3488}
{0: 5901, 1: 1142}
{'No': 3641, 'Yes': 3402}
{'No': 4933, 'Yes': 2110}
{'No': 3390, 'Yes': 2971}
{0: 5174, 1: 1869}
{'type_Month-to-month': 3875, 'type_Two year': 1695, 'type_One year': 1473, 'paperless_billing_Yes': 4171, 'paperless_billing_No': 2872, 'internet_service_Fiber optic': 3096, 'internet_service_DSL': 2421, 'online_security_No': 3498, 'online_security_Yes': 2019, 'online_backup_No': 3088, 'online_backup_Yes': 2429, 'device_protection_No': 3095, 'device_protection_Yes': 2422, 'tech_support_No': 3473, 'tech_support_Yes': 2044, 'streaming_tv_No': 2810, 'streaming_tv_Yes': 2707, 'streaming_movies_No': 2785, 'streaming_movies_Yes': 2732, 'gender_Male': 3555, 'gender_Female': 

In [5]:
# Шаг 4. Сохраним в качестве второго артифакта и сам датафрейм
df.to_csv('users_churn.csv', index=False)

In [6]:
# Шаг 5. Залогируем первые артефакты
import os
import mlflow
# задаём название эксперимента и имя запуска для логирования в MLflow
EXPERIMENT_NAME = "churn_fio"
RUN_NAME = "data_check"

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact("columns.txt", "dataframe")
    mlflow.log_artifact("users_churn.csv", "dataframe")


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id)


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert 'FINISHED' == run.info.status

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt')
os.remove('users_churn.csv')
